In [3]:
import subprocess as sp
import pandas as pd
import numpy as np
from pathlib import Path
import os
working_dir = Path('/d/data/plasmo/gemma')
output_dir = working_dir / 'results'

In [ ]:
df = pd.read_csv((working_dir / 'snps_v3_base.fam'), sep=' ', header=None)

In [ ]:
from sklearn.model_selection import KFold
n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True)
c = 0
for train_index, test_index in kf.split(range(df.shape[0])):
    to_predict = np.random.choice(test_index, n_samples)
    df_mod = df.copy()
    df_mod.loc[to_predict, 5] = "-9"
    df_mod.to_csv((working_dir / 'snps_v3_{0}.fam'.format(c)), sep=' ', header=False, index=False)
    c+= 1

In [11]:
def run_gemma(n):
    prefix = 'snps_v3'
    os.chdir(working_dir)
    output_dir.mkdir(exist_ok=True)
    command_strings = ['rm snps_v3.fam',
                       'rm -r output',
                       'mv snps_v3_{n}.fam snps_v3.fam'.format(n=n),
                       './gemma -bfile {prefix} -gk 2 -o related'.format(prefix=prefix),
                       './gemma -bfile {prefix} -bslmm 2 -o bs'.format(prefix=prefix),
                       './gemma -bfile {prefix} -epm {epm} -emu {emu} -ebv {ebv} -k {k} -predict 1 -o preds'.format(prefix=prefix,
                                                                                                                    epm='output/bs.param.txt',
                                                                                                                    emu='output/bs.log.txt',
                                                                                                                    ebv='output/bs.bv.txt',
                                                                                                                    k='output/related.sXX.txt'),
                      'mv output/preds.prdt.txt results/preds_{n}.txt'.format(n=n)]
    
    for c in command_strings:
        try:
            sp.run(c, shell=True, check=True)
        except Exception as e:
            print(e)
    
for x in range(10):
    run_gemma(x)
    